## Imports

In [1]:
from delta import configure_spark_with_delta_pip
from pyspark.sql import SparkSession
import os
import shutil

## Start Spark Session

In [2]:
# Inicializa uma SparkSession
builder = SparkSession.builder \
    .appName("Criação das tabelas delta") \
    .master("local[*]") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

print(f"\n✅ Spark {spark.version} iniciado!\n")


✅ Spark 3.5.7 iniciado!



## Variables

In [3]:
# Define caminhos locais onde serão armazenadas as tabelas Delta
base_bronze_path = "D:/Projetos/Jornada_financas_pessoais/data/delta/bronze"
base_silver_path = "D:/Projetos/Jornada_financas_pessoais/data/delta/silver"
base_gold_path = "D:/Projetos/Jornada_financas_pessoais/data/delta/gold"

# Define o caminho da tabela Delta Bronze
delta_path_cotahist = f"{base_bronze_path}/raw_cotahist"
delta_path_cadcliente = f"{base_bronze_path}/raw_cadcliente"

# Define o caminho da tabela Delta Silver
delta_path_cotacao_historica = f"{base_silver_path}/stg_cotacao_historica"

# Dimensões
delta_path_dim_ativo = f"{base_gold_path}/dim_ativo_financeiro"
delta_path_dim_tempo = f"{base_gold_path}/dim_tempo"
delta_path_dim_cliente = f"{base_gold_path}/dim_cliente"

# Fato
delta_path_fato_cotacao = f"{base_gold_path}/fato_cotacao"
delta_path_fato_carteira = f"{base_gold_path}/fato_carteira"

## Create Table Bronze

In [ ]:
# Remove o diretório da tabela Delta se ele existir
if os.path.exists(delta_path_cotahist):
    shutil.rmtree(delta_path_cotahist)
    print(f"✅ Diretório removido: {delta_path_cotahist}")

## Create table cotahist
spark.sql(f"""
CREATE TABLE IF NOT EXISTS raw_cotahist (
  tipo_registro STRING,
  data_pregao STRING,
  codigo_bdi STRING,
  codigo_negociacao STRING,
  tipo_mercado STRING,
  nome_resumido_empresa STRING,
  especificacao_papel STRING,
  prazo_dias_mercado STRING,
  moeda_referencia STRING,
  preco_abertura_papel STRING,
  preco_maximo_papel STRING,
  preco_minimo_papel STRING,
  preco_medio_papel STRING,
  preco_ultimo_negocio STRING,
  preco_melhor_oferta_compra STRING,
  preco_melhor_oferta_venda STRING,
  numero_negocios_efetuados STRING,
  quantidade_total_titulos STRING,
  volume_total_titulos STRING,
  preco_exercicio_opcoes STRING,
  indicador_correcao_precos STRING,
  data_vencimento_opcoes STRING,
  fator_cotacao_papel STRING,
  preco_exercicio_pontos STRING,
  codigo_papel_sistema STRING,
  numero_distribuicao_papel STRING
)
USING DELTA
LOCATION '{delta_path_cotahist}'
""")

print("✅ Tabela raw_cotahist criada com sucesso!")

In [4]:
# Remove o diretório da tabela Delta se ele existir
if os.path.exists(delta_path_cadcliente):
    shutil.rmtree(delta_path_cadcliente)
    print(f"✅ Diretório removido: {delta_path_cadcliente}")

# Criar dimensão Cliente
spark.sql(f"""
CREATE TABLE IF NOT EXISTS raw_cadcliente (
    cpf STRING,
    nome STRING,
    data_nascimento STRING,
    email STRING,
    telefone STRING
)
USING DELTA
LOCATION '{delta_path_cadcliente}'
""")

print("✅ Tabela raw_cadcliente criada com sucesso!")

✅ Diretório removido: D:/Projetos/Jornada_financas_pessoais/data/delta/bronze/raw_cadcliente
✅ Tabela raw_cadcliente criada com sucesso!


## Create Table Silver

In [ ]:
# Remove o diretório da tabela Delta se ele existir
if os.path.exists(delta_path_cotacao_historica):
    shutil.rmtree(delta_path_cotacao_historica)
    print(f"✅ Diretório removido: {delta_path_cotacao_historica}")

## Create table stg_cotacao_historica
spark.sql(f"""
CREATE TABLE IF NOT EXISTS stg_cotacao_historica (
  tp_registro STRING COMMENT 'Tipo do registro no arquivo',
  dt_pregao DATE COMMENT 'Data do pregão',
  cd_bdi STRING COMMENT 'Código BDI (identificação da ação)',
  cd_negociacao STRING COMMENT 'Código de negociação do papel',
  tp_mercado STRING COMMENT 'Tipo de mercado',
  nm_empresa STRING COMMENT 'Nome resumido da empresa',
  ds_especificacao_papel STRING COMMENT 'Especificação do papel',
  nr_prazo_dias_mercado STRING COMMENT 'Prazo em dias de mercado',
  cd_moeda_referencia STRING COMMENT 'Moeda de referência',
  vl_abertura DECIMAL(11,2) COMMENT 'Preço de abertura do papel',
  vl_maximo DECIMAL(11,2) COMMENT 'Preço máximo do papel',
  vl_minimo DECIMAL(11,2) COMMENT 'Preço mínimo do papel',
  vl_medio DECIMAL(11,2) COMMENT 'Preço médio do papel',
  vl_ultimo_negocio DECIMAL(11,2) COMMENT 'Preço do último negócio',
  vl_melhor_oferta_compra DECIMAL(11,2) COMMENT 'Preço da melhor oferta de compra',
  vl_melhor_oferta_venda DECIMAL(11,2) COMMENT 'Preço da melhor oferta de venda',
  qt_negocios_efetuados INT COMMENT 'Número de negócios efetuados',
  qt_total_titulos INT COMMENT 'Quantidade total de títulos negociados',
  vl_total_titulos DECIMAL(16,2) COMMENT 'Volume total títulos negociados',
  vl_exercicio_opcoes DECIMAL(11,2) COMMENT 'Preço de exercício de opções',
  cd_indicador_correcao STRING COMMENT 'Indicador de correção de preços',
  dt_vencimento_opcoes DATE COMMENT 'Data de vencimento das opções',
  cd_fator_cotacao STRING COMMENT 'Fator de cotação do papel',
  vl_exercicio_pontos DECIMAL(7,6) COMMENT 'Preço de exercício em pontos',
  cd_papel_sistema STRING COMMENT 'Código do papel no sistema',
  nr_distribuicao_papel STRING COMMENT 'Número de distribuição do papel',
  ts_insercao TIMESTAMP COMMENT 'Timestamp de inserção do registro'
)
USING DELTA
COMMENT 'Histórico de cotações de papéis da bolsa de valores'
LOCATION '{delta_path_cotacao_historica}'
""")

print("✅ Tabela stg_cotacao_historica criada com sucesso!")

## Create Dimension Tables

In [ ]:
# Remove o diretório da tabela Delta se ele existir
if os.path.exists(delta_path_dim_ativo):
    shutil.rmtree(delta_path_dim_ativo)
    print(f"✅ Diretório removido: {delta_path_dim_ativo}")

# Criar dimensão Ativo
spark.sql(f"""
CREATE TABLE IF NOT EXISTS dim_ativo_financeiro (
    sk_ativo STRING NOT NULL COMMENT 'Chave surrogate da dimensão ativo',
    cd_ativo STRING NOT NULL COMMENT 'Código de negociação do ativo',
    nm_empresa STRING COMMENT 'Nome da empresa',
    ds_ativo STRING COMMENT 'Descrição do ativo',
    cd_tipo_mercado STRING COMMENT 'Código do tipo de mercado',
    ds_tipo_mercado STRING COMMENT 'Descrição do tipo de mercado',
    cd_isin STRING COMMENT 'Código ISIN do ativo',
    ds_tipo_ativo STRING COMMENT 'Descrição do tipo do ativo (ação, FII, etc)',
    ts_insercao TIMESTAMP COMMENT 'Timestamp de inserção do registro',
    ts_atualizacao TIMESTAMP COMMENT 'Timestamp da última atualização'
)
USING DELTA
COMMENT 'Dimensão de ativos financeiros'
LOCATION '{delta_path_dim_ativo}'
""")

print("✅ Tabela dim_ativo_financeiro criada com sucesso!")

In [ ]:
# Remove o diretório da tabela Delta se ele existir
if os.path.exists(delta_path_dim_tempo):
    shutil.rmtree(delta_path_dim_tempo)
    print(f"✅ Diretório removido: {delta_path_dim_tempo}")

# Criar dimensão Tempo
spark.sql(f"""
CREATE TABLE IF NOT EXISTS dim_tempo (
    dt_referencia DATE NOT NULL COMMENT 'Data de referência',
    nr_ano INT COMMENT 'Número do ano',
    nr_semestre INT COMMENT 'Número do semestre no ano (1-2)',
    nr_trimestre INT COMMENT 'Número do trimestre no ano (1-4)',
    nr_mes INT COMMENT 'Número do mês no ano (1-12)',
    nm_mes STRING COMMENT 'Nome do mês',
    nr_semana INT COMMENT 'Número da semana no ano',
    nr_dia INT COMMENT 'Número do dia no mês',
    nr_dia_ano INT COMMENT 'Número do dia no ano',
    nr_dia_semana INT COMMENT 'Número do dia na semana',
    nm_dia_semana STRING COMMENT 'Nome do dia da semana',
    fl_feriado BOOLEAN COMMENT 'Flag que indica se é feriado',
    fl_fim_semana BOOLEAN COMMENT 'Flag que indica se é fim de semana',
    fl_dia_util BOOLEAN COMMENT 'Flag que indica se é dia útil'
)
USING DELTA
COMMENT 'Dimensão de tempo'
LOCATION '{delta_path_dim_tempo}'
""")

print("✅ Tabela dim_tempo criada com sucesso!")

In [ ]:
# Remove o diretório da tabela Delta se ele existir
if os.path.exists(delta_path_dim_cliente):
    shutil.rmtree(delta_path_dim_cliente)
    print(f"✅ Diretório removido: {delta_path_dim_cliente}")

# Criar dimensão Cliente
spark.sql(f"""
CREATE TABLE IF NOT EXISTS dim_cliente (
    sk_cliente STRING NOT NULL COMMENT 'Chave surrogate da dimensão cliente',
    cd_cpf_pessoa STRING NOT NULL COMMENT 'Código do cpf da pessoa',
    nm_cliente STRING COMMENT 'Nome do cliente',
    dt_nascimento DATE COMMENT 'Data de nascimento',
    ds_email STRING COMMENT 'Endereço de email',
    nr_telefone STRING COMMENT 'Número de telefone',
    ts_insercao TIMESTAMP COMMENT 'Timestamp de inserção do registro',
    ts_atualizacao TIMESTAMP COMMENT 'Timestamp da última atualização'
)
USING DELTA
COMMENT 'Dimensão de clientes'
LOCATION '{delta_path_dim_cliente}'
""")

print("✅ Tabela dim_cliente criada com sucesso!")

## Create Fact Table

In [ ]:
# Remove diretório existente da tabela fato
if os.path.exists(delta_path_fato_cotacao):
    shutil.rmtree(delta_path_fato_cotacao)
    print(f"✅ Diretório removido: {delta_path_fato_cotacao}")

# Criar tabela fato de cotações
spark.sql(f"""
CREATE TABLE IF NOT EXISTS fato_cotacao (
    dt_pregao DATE NOT NULL COMMENT 'Data do pregão',
    sk_ativo STRING NOT NULL COMMENT 'Chave surrogate da dimensão ativo',
    vl_abertura DECIMAL(11,2) COMMENT 'Preço de abertura',
    vl_minimo DECIMAL(11,2) COMMENT 'Preço mínimo',
    vl_maximo DECIMAL(11,2) COMMENT 'Preço máximo',
    vl_medio DECIMAL(11,2) COMMENT 'Preço médio',
    vl_ultimo_negocio DECIMAL(11,2) COMMENT 'Preço do último negócio',
    qt_negocio INT COMMENT 'Quantidade de negócios realizados',
    qt_titulo INT COMMENT 'Quantidade de títulos negociados',
    vl_volume DECIMAL(16,2) COMMENT 'Volume financeiro negociado',
    ts_insercao TIMESTAMP COMMENT 'Timestamp de inserção do registro',
    ano_pregao INT NOT NULL COMMENT 'Ano do pregão (partição)',
    mes_pregao INT NOT NULL COMMENT 'Mês do pregão (partição)'
)
USING DELTA
PARTITIONED BY (ano_pregao, mes_pregao)
COMMENT 'Fato de cotações de ativos'
LOCATION '{delta_path_fato_cotacao}'
""")

print("✅ Tabela fato_cotacao criada com sucesso!")

In [ ]:
# Remove diretório existente da tabela fato
if os.path.exists(delta_path_fato_carteira):
    shutil.rmtree(delta_path_fato_carteira)
    print(f"✅ Diretório removido: {delta_path_fato_carteira}")

# Criar tabela fato de carteira
spark.sql(f"""
CREATE TABLE IF NOT EXISTS fato_carteira (
    dt_carteira DATE NOT NULL COMMENT 'Data da carteira',
    sk_cliente STRING NOT NULL COMMENT 'Chave surrogate da dimensão cliente',
    sk_ativo STRING NOT NULL COMMENT 'Chave surrogate da dimensão ativo',
    qt_ativo INT COMMENT 'Quantidade de ativos na carteira',
    vl_ativo DECIMAL(11,2) COMMENT 'Preço do ativo',
    vl_medio DECIMAL(11,2) COMMENT 'Preço médio do ativo',
    vl_total_investido DECIMAL(16,2) COMMENT 'Valor total investido',
    vl_total_carteira DECIMAL(16,2) COMMENT 'Valor total da carteira',      
    ts_insercao TIMESTAMP COMMENT 'Timestamp de inserção do registro',
)
USING DELTA
COMMENT 'Fato de carteira de ativos'
LOCATION '{delta_path_fato_carteira}'
""")

print("✅ Tabela fato_carteira criada com sucesso!")

## Stop Spark Session

In [ ]:
# Encerra a SparkSession
spark.stop()